In [1]:
this_file_name = 'test_position_control_class.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))
from utils import *
import pid
from controllers import *

In [2]:
delL_params = np.array([3,2,1])
delTh_params = np.array([-3,-2,-1])
uncoupling_m = np.array([[0.01,1.954],[0.01,-1.954]])

In [3]:
pid(0,1,2)

In [4]:
u_c = UncoupledController(delL_controller_params=delL_params,
                          delTH_controller_params=delTh_params,
                          uncoupling_matrix=uncoupling_m)